In [1]:
import pandas as pd
import datetime
import numpy as np

In [2]:
df = pd.read_csv(
    "https://raw.githubusercontent.com/JackyP/testing/master/datasets/nycflights.csv",
    index_col=0
    )
df.head()

,year,month,day,dep_time,dep_delay,arr_time,arr_delay,carrier,tailnum,flight,origin,dest,air_time,distance,hour,minute
1,2013,1,1,517.0,2.0,830.0,11.0,UA,N14228,1545,EWR,IAH,227.0,1400,5.0,17.0
2,2013,1,1,533.0,4.0,850.0,20.0,UA,N24211,1714,LGA,IAH,227.0,1416,5.0,33.0
3,2013,1,1,542.0,2.0,923.0,33.0,AA,N619AA,1141,JFK,MIA,160.0,1089,5.0,42.0
4,2013,1,1,544.0,-1.0,1004.0,-18.0,B6,N804JB,725,JFK,BQN,183.0,1576,5.0,44.0
5,2013,1,1,554.0,-6.0,812.0,-25.0,DL,N668DN,461,LGA,ATL,116.0,762,5.0,54.0


## Saneamento de Dados

### Seleção de colunas & Tratamento dos nulos

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 336776 entries, 1 to 336776
Data columns (total 16 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   year       336776 non-null  int64  
 1   month      336776 non-null  int64  
 2   day        336776 non-null  int64  
 3   dep_time   328521 non-null  float64
 4   dep_delay  328521 non-null  float64
 5   arr_time   328063 non-null  float64
 6   arr_delay  327346 non-null  float64
 7   carrier    336776 non-null  object 
 8   tailnum    334264 non-null  object 
 9   flight     336776 non-null  int64  
 10  origin     336776 non-null  object 
 11  dest       336776 non-null  object 
 12  air_time   327346 non-null  float64
 13  distance   336776 non-null  int64  
 14  hour       328521 non-null  float64
 15  minute     328521 non-null  float64
dtypes: float64(7), int64(5), object(4)
memory usage: 43.7+ MB


Critérios :
1. ["arr_time", "dep_time", "carrier", "flight"] não podem ser nulos
2. As observações nulas destes campos devem ser removidos
3. Se fosse para tratar os nulos usar: ex df["dep_time"].fillna("NA")

In [4]:
usecols=["dep_time","arr_time","carrier","flight", "tailnum","air_time","distance", "origin", "dest"]

In [5]:
df_raw = df.loc[
    (~df["arr_time"].isna()) \
    & (~df["dep_time"].isna()) \
    & (~df["carrier"].isna()) \
    & (~df["flight"].isna())
].loc[:, usecols]

In [6]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 328063 entries, 1 to 336770
Data columns (total 9 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   dep_time  328063 non-null  float64
 1   arr_time  328063 non-null  float64
 2   carrier   328063 non-null  object 
 3   flight    328063 non-null  int64  
 4   tailnum   328063 non-null  object 
 5   air_time  327346 non-null  float64
 6   distance  328063 non-null  int64  
 7   origin    328063 non-null  object 
 8   dest      328063 non-null  object 
dtypes: float64(3), int64(2), object(4)
memory usage: 25.0+ MB


In [7]:
df_raw["air_time"] = df_raw["air_time"].fillna(0)

In [8]:
df_raw.drop_duplicates(inplace=True)

In [9]:
df_raw = df_raw.astype("object")

Perguntas:
1. Qual a vantagem de avaliar os nulos no começo do processo? Podemos pegar erro logo de início
2. Quais os riscos envolvidos? Se estvermos em um pipeline, perdemos a "foto" do dado bruto para uma camada raw. A boa prática em um pipeline seria realizar apenas a transformação para str / objeto

In [10]:
tmp = df.loc[:, usecols].copy()
for col in ["arr_time", "dep_time", "carrier", "flight"]:
    tmp_df = tmp.loc[~df[col].isna()]
    tmp = tmp_df.copy()

In [11]:
tmp.drop_duplicates(inplace=True)

In [12]:
tmp.shape[0] == df_raw.shape[0]

True

### Rename e tipagem

In [13]:
new_columns = ["datetime_partida", "datetime_chegada", "companhia", "id_voo", "id_aeronave","tempo_voo", "distancia", "origem", "destino"]

In [14]:
columns_map = {usecols[i]: new_columns[i] for i in range(len(usecols))}
columns_map

{'dep_time': 'datetime_partida',
 'arr_time': 'datetime_chegada',
 'carrier': 'companhia',
 'flight': 'id_voo',
 'tailnum': 'id_aeronave',
 'air_time': 'tempo_voo',
 'distance': 'distancia',
 'origin': 'origem',
 'dest': 'destino'}

In [15]:
df_work = df_raw.copy()
df_work.rename(columns=columns_map, inplace=True)
df_work.head()

,datetime_partida,datetime_chegada,companhia,id_voo,id_aeronave,tempo_voo,distancia,origem,destino
1,517.0,830.0,UA,1545,N14228,227.0,1400,EWR,IAH
2,533.0,850.0,UA,1714,N24211,227.0,1416,LGA,IAH
3,542.0,923.0,AA,1141,N619AA,160.0,1089,JFK,MIA
4,544.0,1004.0,B6,725,N804JB,183.0,1576,JFK,BQN
5,554.0,812.0,DL,461,N668DN,116.0,762,LGA,ATL


In [16]:
df_work.dtypes

datetime_partida    object
datetime_chegada    object
companhia           object
id_voo              object
id_aeronave         object
tempo_voo           object
distancia           object
origem              object
destino             object
dtype: object

In [17]:
df_work["tempo_voo"] = df_work.loc[:,"tempo_voo"].astype(float)
df_work["distancia"] = df_work.loc[:,"distancia"].astype(float)
df_work["companhia"] = df_work.loc[:,"companhia"].astype(str)
df_work["id_voo"] = df_work.loc[:,"id_voo"].astype(str)
df_work["id_aeronave"] = df_work.loc[:,"id_aeronave"].astype(str)
df_work["datetime_partida"] = df_work.loc[:,"datetime_partida"].astype(str)
df_work["datetime_chegada"] = df_work.loc[:,"datetime_chegada"].astype(str)
df_work["origem"] = df_work.loc[:,"origem"].astype(str)
df_work["destino"] = df_work.loc[:,"destino"].astype(str)

In [18]:
df_work.dtypes

datetime_partida     object
datetime_chegada     object
companhia            object
id_voo               object
id_aeronave          object
tempo_voo           float64
distancia           float64
origem               object
destino              object
dtype: object

### Tratamento strings

In [19]:
import re

def padroniza_str(obs):
    return re.sub('[^A-Za-z0-9]+', '', obs.upper())

In [20]:
padroniza_str("ahsuw! @ ++  ~ ç 3n!!rrr")

'AHSUW3NRRR'

Guias práticos para regex:
- https://medium.com/@tomstaite1/everything-you-need-to-know-about-regular-expressions-regex-3cbc5b95146
- https://medium.com/xp-inc/regex-um-guia-pratico-para-express%C3%B5es-regulares-1ac5fa4dd39f

In [21]:
df_work["companhia_formatted"] = df_work.loc[:,"companhia"].apply(lambda x: padroniza_str(x))
df_work["id_voo_formatted"] = df_work.loc[:,"id_voo"].apply(lambda x: padroniza_str(x))
df_work["id_aeronave_formatted"] = df_work.loc[:,"id_aeronave"].apply(lambda x: padroniza_str(x))
df_work["origem_formatted"] = df_work.loc[:,"origem"].apply(lambda x: padroniza_str(x))
df_work["destino_formatted"] = df_work.loc[:,"destino"].apply(lambda x: padroniza_str(x))

In [22]:
df_work.head()

,datetime_partida,datetime_chegada,companhia,id_voo,id_aeronave,tempo_voo,distancia,origem,destino,companhia_formatted,id_voo_formatted,id_aeronave_formatted,origem_formatted,destino_formatted
1,517.0,830.0,UA,1545,N14228,227.0,1400.0,EWR,IAH,UA,1545,N14228,EWR,IAH
2,533.0,850.0,UA,1714,N24211,227.0,1416.0,LGA,IAH,UA,1714,N24211,LGA,IAH
3,542.0,923.0,AA,1141,N619AA,160.0,1089.0,JFK,MIA,AA,1141,N619AA,JFK,MIA
4,544.0,1004.0,B6,725,N804JB,183.0,1576.0,JFK,BQN,B6,725,N804JB,JFK,BQN
5,554.0,812.0,DL,461,N668DN,116.0,762.0,LGA,ATL,DL,461,N668DN,LGA,ATL


### Tratamentos especiais
Neste caso, vamos realizar tratamento das horas

In [23]:
df_work.loc[:,"datetime_partida"] = df_work.loc[:,"datetime_partida"].str.replace('.0', '')
df_work.loc[:,"datetime_chegada"] = df_work.loc[:,"datetime_chegada"].str.replace('.0', '')
df_work["data_voo"] = pd.to_datetime(df[['year', 'month', 'day']]) 

C:\Users\RyanO\AppData\Local\Temp\ipykernel_15088\3628829587.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df_work.loc[:,"datetime_partida"] = df_work.loc[:,"datetime_partida"].str.replace('.0', '')
C:\Users\RyanO\AppData\Local\Temp\ipykernel_15088\3628829587.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df_work.loc[:,"datetime_chegada"] = df_work.loc[:,"datetime_chegada"].str.replace('.0', '')


In [24]:
df_work["datetime_partida"].apply(lambda x: len(x)).unique()

array([3, 1, 2, 0, 4], dtype=int64)

In [25]:
df_work.head(20)

,datetime_partida,datetime_chegada,companhia,id_voo,id_aeronave,tempo_voo,distancia,origem,destino,companhia_formatted,id_voo_formatted,id_aeronave_formatted,origem_formatted,destino_formatted,data_voo
1,517,8,UA,1545,N14228,227.0,1400.0,EWR,IAH,UA,1545,N14228,EWR,IAH,2013-01-01
2,533,8,UA,1714,N24211,227.0,1416.0,LGA,IAH,UA,1714,N24211,LGA,IAH,2013-01-01
3,542,923,AA,1141,N619AA,160.0,1089.0,JFK,MIA,AA,1141,N619AA,JFK,MIA,2013-01-01
4,544,04,B6,725,N804JB,183.0,1576.0,JFK,BQN,B6,725,N804JB,JFK,BQN,2013-01-01
5,554,812,DL,461,N668DN,116.0,762.0,LGA,ATL,DL,461,N668DN,LGA,ATL,2013-01-01
6,554,7,UA,1696,N39463,150.0,719.0,EWR,ORD,UA,1696,N39463,EWR,ORD,2013-01-01
7,555,913,B6,507,N516JB,158.0,1065.0,EWR,FLL,B6,507,N516JB,EWR,FLL,2013-01-01
8,557,9,EV,5708,N829AS,53.0,229.0,LGA,IAD,EV,5708,N829AS,LGA,IAD,2013-01-01
9,557,838,B6,79,N593JB,140.0,944.0,JFK,MCO,B6,79,N593JB,JFK,MCO,2013-01-01
10,558,753,AA,301,N3ALAA,138.0,733.0,LGA,ORD,AA,301,N3ALAA,LGA,ORD,2013-01-01


In [26]:
def corrige_hora(hr_str,dct_hora = {1:"000?",2:"00?",3:"0?",4:"?"}):
    if hr_str == "2400":
        return "00:00"
    elif hr_str == "":
        return "00:00"
    elif hr_str == " ":
        return "00:00"
    elif (len(hr_str) == 2) & (int(hr_str) <= 12):
        return f"0{hr_str[0]}:{hr_str[1]}0"
    else:
        hora = dct_hora[len(hr_str)].replace("?", hr_str)
        return f"{hora[:2]}:{hora[2:]}"

In [27]:
datetime_partida2 = df_work.loc[:,"datetime_partida"].apply(lambda x: corrige_hora(x))
datetime_chegada2 = df_work.loc[:,"datetime_chegada"].apply(lambda x: corrige_hora(x))

In [28]:
datetime_partida2[119:122]

120    08:11
121    08:11
122    08:12
Name: datetime_partida, dtype: object

In [29]:
def fix_minutes(time_str):
    pattern = r'(\d{2}):([6-9]\d)'
    return re.sub(pattern, r'\1:59', time_str)

In [30]:
datetime_partida2 = datetime_partida2.apply(lambda x: fix_minutes(x))
datetime_chegada2 = datetime_chegada2.apply(lambda x: fix_minutes(x))

In [31]:
pd.to_datetime(df_work.loc[:,'data_voo'].astype(str) + " " + datetime_partida2)

1        2013-01-01 05:17:00
2        2013-01-01 05:33:00
3        2013-01-01 05:42:00
4        2013-01-01 05:44:00
5        2013-01-01 05:54:00
                 ...        
336766   2013-09-30 00:22:00
336767   2013-09-30 00:22:00
336768   2013-09-30 22:41:00
336769   2013-09-30 00:27:00
336770   2013-09-30 23:49:00
Length: 328038, dtype: datetime64[ns]

In [32]:
df_work['datetime_partida_formatted'] = pd.to_datetime(df_work.loc[:,'data_voo'].astype(str) + " " + datetime_partida2)
df_work['datetime_chegada_formatted'] = pd.to_datetime(df_work.loc[:,'data_voo'].astype(str) + " " + datetime_chegada2)

In [33]:
df_work.head()

,datetime_partida,datetime_chegada,companhia,id_voo,id_aeronave,tempo_voo,distancia,origem,destino,companhia_formatted,id_voo_formatted,id_aeronave_formatted,origem_formatted,destino_formatted,data_voo,datetime_partida_formatted,datetime_chegada_formatted
1,517,8,UA,1545,N14228,227.0,1400.0,EWR,IAH,UA,1545,N14228,EWR,IAH,2013-01-01,2013-01-01 05:17:00,2013-01-01 00:08:00
2,533,8,UA,1714,N24211,227.0,1416.0,LGA,IAH,UA,1714,N24211,LGA,IAH,2013-01-01,2013-01-01 05:33:00,2013-01-01 00:08:00
3,542,923,AA,1141,N619AA,160.0,1089.0,JFK,MIA,AA,1141,N619AA,JFK,MIA,2013-01-01,2013-01-01 05:42:00,2013-01-01 09:23:00
4,544,04,B6,725,N804JB,183.0,1576.0,JFK,BQN,B6,725,N804JB,JFK,BQN,2013-01-01,2013-01-01 05:44:00,2013-01-01 00:40:00
5,554,812,DL,461,N668DN,116.0,762.0,LGA,ATL,DL,461,N668DN,LGA,ATL,2013-01-01,2013-01-01 05:54:00,2013-01-01 08:12:00


In [34]:
df_work["datetime_chegada_formatted"] = np.where(
    df_work["datetime_partida_formatted"] > df_work["datetime_chegada_formatted"],
    df_work["datetime_chegada_formatted"] + pd.Timedelta(days=1),
    df_work["datetime_chegada_formatted"]
    )

In [35]:
df_work.loc[df_work["datetime_partida_formatted"] > df_work["datetime_chegada_formatted"]]

,datetime_partida,datetime_chegada,companhia,id_voo,id_aeronave,tempo_voo,distancia,origem,destino,companhia_formatted,id_voo_formatted,id_aeronave_formatted,origem_formatted,destino_formatted,data_voo,datetime_partida_formatted,datetime_chegada_formatted


In [36]:
df_dw = df_work[["data_voo",
         "companhia_formatted",
         "id_voo_formatted",
         "id_aeronave_formatted",
         "datetime_partida_formatted",
         "datetime_chegada_formatted",
         "origem_formatted",
         "destino_formatted",
         "tempo_voo",
         "distancia"]].copy()

## Engenharia de Features

- tempo_voo_esperado
- tempo_voo_hr
- atraso (em horas)
- flg_status
- dia_semana
- horario

In [37]:
df_dw["tempo_voo_esperado"] = (df_dw["datetime_chegada_formatted"] - df_dw["datetime_partida_formatted"]) / pd.Timedelta(hours=1)
df_dw["tempo_voo_hr"] = df_dw["tempo_voo"] /60
df_dw["atraso"] = df_dw["tempo_voo_hr"] - df_dw["tempo_voo_esperado"]
df_dw["dia_semana"] = df_dw["data_voo"].dt.day_of_week #0=segunda

In [38]:
def classifica_hora(hra):
    if 0 <= hra < 6: return "MADRUGADA"
    elif 6 <= hra < 12: return "MANHA"
    elif 12 <= hra < 18: return "TARDE"
    else: return "NOITE"

In [39]:
df_dw["horario"] = df_work.loc[:,"datetime_partida_formatted"].dt.hour.apply(lambda x: classifica_hora(x))

In [40]:
df_work.loc[:,"datetime_partida_formatted"].dt.hour.apply(lambda x: classifica_hora(x))

1         MADRUGADA
2         MADRUGADA
3         MADRUGADA
4         MADRUGADA
5         MADRUGADA
            ...    
336766    MADRUGADA
336767    MADRUGADA
336768        NOITE
336769    MADRUGADA
336770        NOITE
Name: datetime_partida_formatted, Length: 328038, dtype: object

In [41]:
df_dw.head()

,data_voo,companhia_formatted,id_voo_formatted,id_aeronave_formatted,datetime_partida_formatted,datetime_chegada_formatted,origem_formatted,destino_formatted,tempo_voo,distancia,tempo_voo_esperado,tempo_voo_hr,atraso,dia_semana,horario
1,2013-01-01,UA,1545,N14228,2013-01-01 05:17:00,2013-01-02 00:08:00,EWR,IAH,227.0,1400.0,18.850000,3.783333,-15.066667,1,MADRUGADA
2,2013-01-01,UA,1714,N24211,2013-01-01 05:33:00,2013-01-02 00:08:00,LGA,IAH,227.0,1416.0,18.583333,3.783333,-14.800000,1,MADRUGADA
3,2013-01-01,AA,1141,N619AA,2013-01-01 05:42:00,2013-01-01 09:23:00,JFK,MIA,160.0,1089.0,3.683333,2.666667,-1.016667,1,MADRUGADA
4,2013-01-01,B6,725,N804JB,2013-01-01 05:44:00,2013-01-02 00:40:00,JFK,BQN,183.0,1576.0,18.933333,3.050000,-15.883333,1,MADRUGADA
5,2013-01-01,DL,461,N668DN,2013-01-01 05:54:00,2013-01-01 08:12:00,LGA,ATL,116.0,762.0,2.300000,1.933333,-0.366667,1,MADRUGADA


In [42]:
print(
    len(df_dw[df_dw["atraso"] > 5]),
    len(df_dw[df_dw["atraso"] > 6])
    )

2545 112


In [43]:
df_dw[df_dw["atraso"]>-1].describe()

,tempo_voo,distancia,tempo_voo_esperado,tempo_voo_hr,atraso,dia_semana
count,165504.000000,165504.000000,165504.000000,165504.000000,165504.000000,165504.000000
mean,153.122354,1067.572754,2.017217,2.552039,0.534822,2.897441
std,96.157011,755.013072,1.091155,1.602617,1.365854,1.992520
min,0.000000,80.000000,0.000000,0.000000,-1.000000,0.000000
25%,83.000000,529.000000,1.233333,1.383333,-0.450000,1.000000
50%,131.000000,937.000000,2.000000,2.183333,-0.233333,3.000000
75%,196.000000,1400.000000,2.800000,3.266667,1.316667,5.000000
max,695.000000,4983.000000,11.633333,11.583333,11.016667,6.000000


In [44]:
df_filtrada = df_dw[df_dw["atraso"]>-1].copy()

In [45]:
def flg_status(atraso):
    if atraso > 0.5 : return "ATRASO"
    else: return "ONTIME"

In [46]:
df_filtrada["flg_status"] = df_filtrada.loc[:,"atraso"].apply(lambda x: flg_status(x))

In [47]:
df_filtrada["flg_status"].value_counts()

ONTIME    99496
ATRASO    66008
Name: flg_status, dtype: int64

In [48]:
df_filtrada

,data_voo,companhia_formatted,id_voo_formatted,id_aeronave_formatted,datetime_partida_formatted,datetime_chegada_formatted,origem_formatted,destino_formatted,tempo_voo,distancia,tempo_voo_esperado,tempo_voo_hr,atraso,dia_semana,horario,flg_status
5,2013-01-01,DL,461,N668DN,2013-01-01 05:54:00,2013-01-01 08:12:00,LGA,ATL,116.0,762.0,2.300000,1.933333,-0.366667,1,MADRUGADA,ONTIME
7,2013-01-01,B6,507,N516JB,2013-01-01 05:55:00,2013-01-01 09:13:00,EWR,FLL,158.0,1065.0,3.300000,2.633333,-0.666667,1,MADRUGADA,ONTIME
9,2013-01-01,B6,79,N593JB,2013-01-01 05:57:00,2013-01-01 08:38:00,JFK,MCO,140.0,944.0,2.683333,2.333333,-0.350000,1,MADRUGADA,ONTIME
10,2013-01-01,AA,301,N3ALAA,2013-01-01 05:58:00,2013-01-01 07:53:00,LGA,ORD,138.0,733.0,1.916667,2.300000,0.383333,1,MADRUGADA,ONTIME
11,2013-01-01,B6,49,N793JB,2013-01-01 05:58:00,2013-01-01 08:49:00,JFK,PBI,149.0,1028.0,2.850000,2.483333,-0.366667,1,MADRUGADA,ONTIME
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
336763,2013-09-30,UA,471,N578UA,2013-09-30 22:33:00,2013-10-01 01:12:00,EWR,SFO,318.0,2565.0,2.650000,5.300000,2.650000,0,NOITE,ATRASO
336764,2013-09-30,B6,1083,N804JB,2013-09-30 22:35:00,2013-10-01 00:59:00,JFK,MCO,123.0,944.0,2.400000,2.050000,-0.350000,0,NOITE,ONTIME
336765,2013-09-30,B6,234,N318JB,2013-09-30 22:37:00,2013-09-30 23:45:00,JFK,BTV,43.0,266.0,1.133333,0.716667,-0.416667,0,NOITE,ONTIME
336768,2013-09-30,B6,486,N346JB,2013-09-30 22:41:00,2013-09-30 23:45:00,JFK,ROC,47.0,264.0,1.066667,0.783333,-0.283333,0,NOITE,ONTIME


In [49]:
df_filtrada.to_csv("data/nycflights_tratada.csv", index=False)